<a href="https://colab.research.google.com/github/sallytakeda/python-excel-kadai/blob/main/auto_order_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import pandas as pd
import datetime

In [9]:
order_files = ["order_A_20230524.xlsx", "order_B_20230524.xlsx", "order_C_20230524.xlsx", "order_D_20230524.xlsx"]

# 野菜ごとの注文合計を計算
order_total = pd.Series(dtype=float)
for file in order_files:
    # Skip the header row by setting header=1
    df = pd.read_excel(file, index_col=0, header=1)
    # Access the second column by position (assuming it contains the order quantities)
    order_quantities = df.iloc[:, 1]
    # Convert the column to numeric, coercing errors to NaN
    order_total = order_total.add(pd.to_numeric(order_quantities, errors='coerce'), fill_value=0)

order_total = order_total.groupby(order_total.index).sum()

In [10]:
inventory_df = pd.read_excel("inventory.xlsx")
latest_inventory = inventory_df.iloc[-1, 2:]  # 日付・曜日を除いた野菜列

In [11]:
updated_inventory = latest_inventory.subtract(order_total, fill_value=0)

In [12]:
pickup_df = pd.read_excel("pickup.xlsx", index_col=0)
thresholds = pickup_df.loc["しきい値"]
replenishment = pickup_df.loc["追加量"]

# 発注が必要な野菜を抽出
needs_order = updated_inventory[updated_inventory < thresholds.reindex(updated_inventory.index)]
order_quantities = replenishment[needs_order.index]

In [13]:
def create_order_email(order_items):
    today = datetime.date.today().strftime("%Y/%m/%d")
    lines = [f"{item}：{int(qty)}個" for item, qty in order_items.items()]
    body = f"""
件名：{today} 野菜発注のご連絡

以下の野菜について、在庫がしきい値を下回ったため、発注をお願いいたします。

{chr(10).join(lines)}

何卒よろしくお願いいたします。
"""
    return body

email_text = create_order_email(order_quantities.to_dict())
print(email_text)


件名：2025/09/20 野菜発注のご連絡

以下の野菜について、在庫がしきい値を下回ったため、発注をお願いいたします。



何卒よろしくお願いいたします。



In [14]:
new_row = [datetime.date.today(), datetime.date.today().strftime("%a")] + updated_inventory.tolist()
inventory_df.loc[len(inventory_df)] = new_row
inventory_df.to_excel("inventory.xlsx", index=False)

In [15]:
{
  "Name": "RunPythonScript",
  "CreationDate": "2025-09-16",
  "Commands": [
    {
      "Command": "open",
      "Target": "file:///C:/Users/Saori/Documents/auto_order.py",
      "Value": ""
    },
    {
      "Command": "run",
      "Target": "cmd",
      "Value": "/c python C:\\Users\\Saori\\Documents\\auto_order.py"
    }
  ]
}

{'Name': 'RunPythonScript',
 'CreationDate': '2025-09-16',
 'Commands': [{'Command': 'open',
   'Target': 'file:///C:/Users/Saori/Documents/auto_order.py',
   'Value': ''},
  {'Command': 'run',
   'Target': 'cmd',
   'Value': '/c python C:\\Users\\Saori\\Documents\\auto_order.py'}]}

In [16]:
!@echo off
!cd "C:\Users\Saori\Documents\UIVision"
!start "" "C:\Program Files\Google\Chrome\Application\chrome.exe" "file:///C:/Users/Saori/Documents/UIVision/RunPythonScript.html"

/bin/bash: line 1: @echo: command not found
/bin/bash: line 1: cd: C:\Users\Saori\Documents\UIVision: No such file or directory
/bin/bash: line 1: start: command not found
